In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession. \
            builder. \
            master('local'). \
            appName("practice"). \
            getOrCreate()

In [5]:
spark

In [45]:
help(spark.read.csv)

Help on method csv in module pyspark.sql.readwriter:

csv(path, schema=None, sep=None, encoding=None, quote=None, escape=None, comment=None, header=None, inferSchema=None, ignoreLeadingWhiteSpace=None, ignoreTrailingWhiteSpace=None, nullValue=None, nanValue=None, positiveInf=None, negativeInf=None, dateFormat=None, timestampFormat=None, maxColumns=None, maxCharsPerColumn=None, maxMalformedLogPerPartition=None, mode=None, columnNameOfCorruptRecord=None, multiLine=None, charToEscapeQuoteEscaping=None, samplingRatio=None, enforceSchema=None, emptyValue=None, locale=None, lineSep=None, pathGlobFilter=None, recursiveFileLookup=None, modifiedBefore=None, modifiedAfter=None, unescapedQuoteHandling=None) method of pyspark.sql.readwriter.DataFrameReader instance
    Loads a CSV file and returns the result as a  :class:`DataFrame`.
    
    This function will go through the input once to determine the input schema if
    ``inferSchema`` is enabled. To avoid going through the entire data once, di

There are three modes we can use while readind csv file into dataframe.

1)PERMISSIVE.
2)DROPMALFORMED.
3)FAILFAST .

1)PERMISSIVE - It is defaule, This mode will put null value if data does not match with schema
.
2)DROPMALFORMED - This mode will not read the corrupt row
.
3)FAILFAST - This mode will throw an exception if csv file has corrupt data

DATA IN CSV FILE

In [ ]:
type         country   city       engines    first_flight   number_built
Airbus A220  Canada    Calgary     2         2013-03-02       179
Airbus A220  Canada    Calgary    two        2013-03-02       179
Airbus A220  Canada    Calgary     2         2013-03-02       179
Airbus A320  France    Lyon       two        1986-06-10      10066
Airbus A330  France    Lyon       two        1992-01-02       1521
Boeing 737   USA       New York   two        1967-08-03      10636
Boeing 737   USA       New York   two        1967-08-03      10636
Boeing 737   USA       New York    2         1967-08-03      10636
Airbus A220  Canada    Calgary     2         2013-03-02       179

In [8]:
path = r"C:\Users\Kshitij kadu\Downloads\sample_csv_corrupt.csv"

In [22]:
schema = """ type string, country string, city string, engines int, first_flight date, number_built int"""

read data using mode = PERMISSIVE

In [30]:
df = spark.read.csv(path, header=True, schema=schema)

In [37]:
df.show()

+-----------+-------+--------+-------+------------+------------+
|       type|country|    city|engines|first_flight|number_built|
+-----------+-------+--------+-------+------------+------------+
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
|Airbus A220| Canada| Calgary|   null|  2013-03-02|         179|
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
|Airbus A320| France|    Lyon|   null|  1986-06-10|       10066|
|Airbus A330| France|    Lyon|   null|  1992-01-02|        1521|
| Boeing 737|    USA|New York|   null|  1967-08-03|       10636|
| Boeing 737|    USA|New York|   null|  1967-08-03|       10636|
| Boeing 737|    USA|New York|      2|  1967-08-03|       10636|
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
+-----------+-------+--------+-------+------------+------------+



read data using mode = DROPMALFORMED

In [38]:
df2 = spark.read.csv(path, header=True, schema=schema, mode = 'DROPMALFORMED')

In [40]:
df2.show()

+-----------+-------+--------+-------+------------+------------+
|       type|country|    city|engines|first_flight|number_built|
+-----------+-------+--------+-------+------------+------------+
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
| Boeing 737|    USA|New York|      2|  1967-08-03|       10636|
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
+-----------+-------+--------+-------+------------+------------+



read data using mode = FAILFAST

In [43]:
df3 = spark.read.csv(path, header=True, schema=schema, mode = 'FAILFAST')

In [44]:
df3.show()

Py4JJavaError: An error occurred while calling o153.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 19) (host.docker.internal executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:400)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$a

If we want to get corrupt data in a separate column then we neew to add '_corrupt_record' in schema
and columnNameOfCorruptRecord = '_corrupt_record' in spark.read

In [48]:
schema = """ type string, country string, city string, engines int, first_flight date,
             number_built int, _corrupt_record string"""

In [49]:
df = spark.read.csv(path, header=True, schema=schema, columnNameOfCorruptRecord = '_corrupt_record' )

In [75]:
df.show(truncate = False)

+-----------+-------+--------+-------+------------+------------+---------------------------------------------+
|type       |country|city    |engines|first_flight|number_built|_corrupt_record                              |
+-----------+-------+--------+-------+------------+------------+---------------------------------------------+
|Airbus A220|Canada |Calgary |2      |2013-03-02  |179         |null                                         |
|Airbus A220|Canada |Calgary |null   |2013-03-02  |179         |Airbus A220,Canada,Calgary,two,2013-03-02,179|
|Airbus A220|Canada |Calgary |2      |2013-03-02  |179         |null                                         |
|Airbus A320|France |Lyon    |null   |1986-06-10  |10066       |Airbus A320,France,Lyon,two,1986-06-10,10066 |
|Airbus A330|France |Lyon    |null   |1992-01-02  |1521        |Airbus A330,France,Lyon,two,1992-01-02,1521  |
|Boeing 737 |USA    |New York|null   |1967-08-03  |10636       |Boeing 737,USA,New York,two,1967-08-03,10636 |
|

In [64]:
df = spark.read.csv(path, header=True, schema=schema, columnNameOfCorruptRecord = '_corrupt_record',badrecordspath = r'C:\Users\Kshitij kadu\Notebooks\temp\badrecord'  )

TypeError: csv() got an unexpected keyword argument 'badrecordspath'

In [ ]:
badRecordsPath = 'C:\Users\Kshitij kadu\Notebooks\temp\badrecord'

In [71]:
spark.read.option('badRecordsPath',r'C:\Users\Kshitij kadu\Notebooks\temp\badrecord\first' ).csv(path, header=True).show()

+-----------+-------+--------+-------+------------+------------+
|       type|country|    city|engines|first_flight|number_built|
+-----------+-------+--------+-------+------------+------------+
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
|Airbus A220| Canada| Calgary|    two|  2013-03-02|         179|
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
|Airbus A320| France|    Lyon|    two|  1986-06-10|       10066|
|Airbus A330| France|    Lyon|    two|  1992-01-02|        1521|
| Boeing 737|    USA|New York|    two|  1967-08-03|       10636|
| Boeing 737|    USA|New York|    two|  1967-08-03|       10636|
| Boeing 737|    USA|New York|      2|  1967-08-03|       10636|
|Airbus A220| Canada| Calgary|      2|  2013-03-02|         179|
+-----------+-------+--------+-------+------------+------------+

